In [ ]:
import os

import threading
from threading import Lock

import numpy as np

from tensorflow.keras.models import load_model

from autoencoder import Autoencoder
from helper_funcs import *

import pandas
pandas.set_option('display.max_rows', None)

from params import kmeans_test

In [ ]:
models = os.listdir('./models/')

dataset = b'MNIST/input.dat'
query   = b'MNIST/query.dat'

model_to_files = {}
for i, model in enumerate(models):
    normalized_dataset = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_dataset.dat'
    normalized_query   = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_query.dat'
    encoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_dataset.dat'
    encoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_query.dat'
    decoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_decoded_dataset.dat'
    decoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_decoded_query.dat'

    model_to_files.update({models[i] : [normalized_dataset, normalized_query,
                                        encoded_dataset, encoded_query,
                                        decoded_dataset, decoded_query]})

n = 60000

In [ ]:
for model in model_to_files:
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]

    model = b'models/' + model.encode()

    # load model
    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # get shape of encoded layer

    # load dataset
    x_train = load_dataset(dataset)
    x_train = x_train.astype('float32') / 255.
    x_test = load_dataset(query)
    x_test = x_test.astype('float32') / 255.
    if len(shape) == 3: # if model type is convolutional
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
    else:
        x_train = np.reshape(x_train, (len(x_train), 784))
        x_test = np.reshape(x_test, (len(x_test), 784))

    encoded_train = autoencoder.encode(x_train)
    encoded_test = autoencoder.encode(x_test)

    # deflatten encoded datasets
    encoded_train = deflatten_encoded(encoded_train, shape)
    encoded_test = deflatten_encoded(encoded_test, shape)

    # decode encoded datasets
    decoded_train = autoencoder.decode(encoded_train)
    decoded_test = autoencoder.decode(encoded_test)

    # save original datasets normalized
    save_decoded_binary(x_train, normalized_dataset)
    save_decoded_binary(x_test, normalized_query)

    # normalize encoded datasets
    encoded_train = normalize(encoded_train)
    encoded_test = normalize(encoded_test)

    # save encoded datasets
    save_encoded_binary(encoded_train, encoded_dataset)
    save_encoded_binary(encoded_test, encoded_query)

    # normalize decoded datasets
    decoded_train = normalize(decoded_train)
    decoded_test = normalize(decoded_test)

    # save decoded datasets
    save_decoded_binary(decoded_train, decoded_dataset)
    save_decoded_binary(decoded_test, decoded_query)

In [ ]:
best_params_lsh = [4, 7, 0.6, 1]            # L, k, window, limit_queries
best_params_hypercube = [67, 3, 1000, 0.42] # M, k, probes, window

print_lock = Lock()
rows_lock = Lock()
rows = []

In [ ]:
def print_results(clustering_time, stotal_latent, silhouette):
    print_lock.acquire()
    print("Clustering time :", clustering_time)
    print("Total silhouette:", stotal_latent)
    print("Silhouette per cluster:", silhouette)
    print("-------------------------")
    print_lock.release()

def rows_append(row):
    rows_lock.acquire()
    rows.append(row)
    rows_lock.release()

def run_kmeans_classic(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]

    config = {
        'model': b'CLASSIC',
        'vals': [],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_dataset': encoded_dataset,
        'decoded_dataset': decoded_dataset,
    }

    clustering_time, stotal_latent, silhouette = kmeans_test(conf=config, int_data=0)

    print_results(clustering_time.val, stotal_latent.val, silhouette.val)

    rows_append([model, 'CLASSIC', clustering_time.value, stotal_latent.value, silhouette.val])

def run_kmeans_lsh(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]

    config = {
        'model': b'LSH',
        'vals': best_params_lsh[:-2].append(best_params_lsh[-1]),
        'window': best_params_lsh[-2],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_dataset': encoded_dataset,
        'decoded_dataset': decoded_dataset,
    }

    clustering_time, stotal_latent, silhouette = kmeans_test(conf=config, int_data=0)

    print_results(clustering_time.val, stotal_latent.val, silhouette.val)

    rows_append([model, 'LSH', clustering_time.value, stotal_latent.value, silhouette.val])

def run_kmeans_cube(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]

    config = {
        'model': b'LSH',
        'vals': best_params_hypercube[:-1],
        'window': best_params_hypercube[-1],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_dataset': encoded_dataset,
        'decoded_dataset': decoded_dataset,
    }

    clustering_time, stotal_latent, silhouette = kmeans_test(conf=config, int_data=0)

    print_results(clustering_time.val, stotal_latent.val, silhouette.val)

    rows_append([model, 'CUBE', clustering_time.value, stotal_latent.value, silhouette.val])

In [ ]:
for model in models:
    t_classic = threading.Thread(target=run_kmeans_classic, args=model)
    t_lsh     = threading.Thread(target=run_kmeans_lsh, args=model)
    t_cube    = threading.Thread(target=run_kmeans_cube, args=model)

    t_classic.start()
    t_lsh.start()
    t_cube.start()

    t_classic.join()
    t_lsh.join()
    t_cube.join()

In [ ]:
col_models, col_methods, col_clustering_time, col_stotal_latent = [], [], [], []

for row in rows:
    model, method, clustering_time, stotal_latent, silhouette = row

    col_models.append(model)
    col_methods.append(method)
    col_clustering_time.append(clustering_time)
    col_stotal_latent.append(stotal_latent)

col_dict = {'model': col_models, 'method': col_methods,
            'clustering time': col_clustering_time, 'silhouette total': col_stotal_latent}

df = pandas.DataFrame(data=col_dict)
df